# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2021-11-25	scCODA is a Bayesian model for compositional single-cell data analysis	Nature Communications	Compositional changes of cell types are main drivers of biological processes. Their detection through single-cell experiments is difficult due to the compositionality of the data and low sample sizes. We introduce scCODA ( https://github.com/theislab/scCODA ), a Bayesian model addressing these issues enabling the study of complex cell type effects in disease, and other stimuli. scCODA demonstrated excellent detection performance, while reliably controlling for false discoveries, and identified experimentally verified cell type changes that were missed in original analyses.	Büttner, M., Ostner, J., Müller, C. L., Theis, F. J., and Schubert, B. (2021). "scCODA is a Bayesian model for compositional single-cell data analysis" <i>Nat. Commun.</i>  12, 6876.	scCODA	https://www.nature.com/articles/s41467-021-27150-6
2021-12-07	tascCOD

## Import pandas

We are using the very handy pandas library for dataframes.

In [3]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [4]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2021-11-25,scCODA is a Bayesian model for compositional s...,Nature Communications,Compositional changes of cell types are main d...,"Büttner, M., Ostner, J., Müller, C. L., Theis,...",scCODA,https://www.nature.com/articles/s41467-021-271...
1,2021-12-07,tascCODA: Bayesian Tree-Aggregated Analysis of...,Frontiers in Genetics,Accurate generative statistical modeling of co...,"Ostner, J., Carcy, S., and Müller, C. L. (2021...",tascCODA,https://www.frontiersin.org/articles/10.3389/f...
2,2023-03-31,Best practices for single-cell analysis across...,Nature reviews. Genetics,Recent advances in single-cell technologies ha...,"Heumos, L., Schaar, A. C., Lance, C., Litinets...",best_practices,https://www.nature.com/articles/s41576-023-005...
3,2024-01-23,MetaIBS - large-scale amplicon-based meta anal...,BioRxiv,Irritable Bowel Syndrome (IBS) is a chronic fu...,"Carcy, S., Ostner, J., Tran, V., Menden, M., a...",metaIBS,https://www.biorxiv.org/content/10.1101/2024.0...
4,2024-06-27,BacSC: A general workflow for bacterial single...,BioRxiv,Bacterial single-cell RNA sequencing has the p...,"Ostner, J., Kirk, T., Olayo-Alarcon, R., Thömi...",BacSC,http://biorxiv.org/lookup/doi/10.1101/2024.06....
5,2024-08-07,Pertpy: an end-to-end framework for perturbati...,BioRxiv,Advances in single-cell technology have enable...,"Heumos, L., Ji, Y., May, L., Green, T., Zhang,...",pertpy,https://www.biorxiv.org/content/10.1101/2024.0...
6,2024-12-09,Score matching for differential abundance test...,BioRxiv,"The class of a-b power interaction models, pro...","Ostner, J., Li, H., and Müller, C. L. (2024). ...",cosmoDA,https://www.biorxiv.org/content/10.1101/2024.1...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [5]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [7]:
!ls ../_publications/

2021-11-25-scCODA.md         2024-06-27-BacSC.md
2021-12-07-tascCODA.md       2024-08-07-pertpy.md
2023-03-31-best_practices.md 2024-12-09-cosmoDA.md
2024-01-23-metaIBS.md


In [10]:
!cat ../_publications/2024-12-09-cosmoDA.md

---
title: "Score matching for differential abundance testing of compositional high-throughput sequencing data"
collection: publications
permalink: /publication/2024-12-09-cosmoDA
excerpt: 'The class of a-b power interaction models, proposed by Yu et al. (2024), provides a general framework for modeling sparse compositional count data with pairwise feature interactions. This class includes many distributions as special cases and enables modeling of zero counts through power transformations, making it particularly suitable for modern high-throughput sequencing data with excess zeros, including single-cell RNA-Seq and microbial amplicon data. Here, we present an extension of this class of models that allows inclusion of covariate information, thus enabling accurate characterization of covariate dependencies in heterogeneous populations. Combining this model with a tailored differential abundance (DA) test leads to a novel DA testing scheme, cosmoDA, that can reduce false positive detecti